In [1]:
import gymnasium as gym
NUM_ENVS = 3
envs = gym.vector.make('ALE/Breakout-v5', num_envs=NUM_ENVS)#, render_mode='human')

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [2]:
import tensorflow as tf

# Define the CNN model
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(210, 160, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    return model

# Create the model
model = create_model()
optimizer = tf.keras.optimizers.Adam()
# Copy the model
target_network = tf.keras.models.clone_model(model)



In [15]:
from collections import deque
import numpy as np
import random
import imageio

max_steps = 1_000_000
N = 10
K = 10
EPSILON = 0.2
MINI_BATCH_SIZE = 10
GAMMA = 0.5
TAU = 0.9
converged = False

# Create the replay buffer
buffer_size = 10000  # Define the size of the replay buffer
buffer = deque(maxlen=buffer_size)

observation, info = envs.reset()
print(observation.shape)



while not converged and max_steps > 0:
    print(f"{max_steps=}")
    if len(buffer) > 1:
        print("AA", loss_value)
        # Save the frames as an MP4 video
        output_path = 'output.mp4'
        imageio.mimwrite(output_path, [o for o, _, _, _ in buffer], fps=5, macro_block_size=None)
    target_network.set_weights((1 - TAU) * np.array(target_network.get_weights()) + TAU * np.array(model.get_weights()))
    for n in range(N):
        print(f"n: {n}", loss_value)
        # sample s, a, r, s' and add it to replay buffer
        action = np.argmax(model(observation), axis=0)
        action = [random.choice(range(4)) if random.random() < EPSILON else a for a in action]
        new_observation, reward, terminated, truncated, info = envs.step(action)
        if terminated.any() or truncated.any():
            observation, info = envs.reset()
        # extract individual environments
        for j in range(NUM_ENVS):
            buffer.append((observation[j], action[j] , reward[j], new_observation[j]))
        observation = new_observation

        for k in range(K):
            # sample minibatch from batch
            mini_batch = random.sample(buffer, MINI_BATCH_SIZE if MINI_BATCH_SIZE<=len(buffer) else len(buffer))
            mb_observations = []
            mb_actions = []
            mb_rewards = []
            mb_new_observations = []
            for o, a, r, no in mini_batch:
                mb_observations.append(o)
                mb_actions.append(a)
                mb_rewards.append(r)
                mb_new_observations.append(no)
            # Convert lists to numpy arrays or TensorFlow tensors
            mb_observations = np.array(mb_observations)
            mb_actions = np.array(mb_actions)
            mb_rewards = np.array(mb_rewards)
            mb_new_observations = np.array(mb_new_observations)
            # obtain target Q-values (labels)
            target_q_values = GAMMA * np.max(target_network(mb_new_observations), axis=1) + mb_rewards
            # Perform forward and backward pass
            with tf.GradientTape() as tape:
                predictions = model(mb_observations)
                selected_q_values = tf.reduce_sum(predictions * tf.one_hot(mb_actions, depth=4), axis=1)
                loss_value = tf.reduce_mean((target_q_values - selected_q_values) ** 2)
            gradients = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
    max_steps -= 1




envs.close()

(3, 210, 160, 3)
max_steps=1000000
n: 0 tf.Tensor(0.25, shape=(), dtype=float32)


/var/folders/mt/0n8phlt50rn7h35xhgh97mw80000gn/T/ipykernel_30589/3025377550.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  target_network.set_weights((1 - TAU) * np.array(target_network.get_weights()) + TAU * np.array(model.get_weights()))


n: 1 tf.Tensor(0.25, shape=(), dtype=float32)
n: 2 tf.Tensor(0.25, shape=(), dtype=float32)
n: 3 tf.Tensor(0.25, shape=(), dtype=float32)
n: 4 tf.Tensor(0.25, shape=(), dtype=float32)
n: 5 tf.Tensor(0.25, shape=(), dtype=float32)
n: 6 tf.Tensor(0.25, shape=(), dtype=float32)
n: 7 tf.Tensor(0.25, shape=(), dtype=float32)
n: 8 tf.Tensor(0.25, shape=(), dtype=float32)
n: 9 tf.Tensor(0.25, shape=(), dtype=float32)
max_steps=999999
AA tf.Tensor(0.25, shape=(), dtype=float32)
n: 0 tf.Tensor(0.25, shape=(), dtype=float32)


/var/folders/mt/0n8phlt50rn7h35xhgh97mw80000gn/T/ipykernel_30589/3025377550.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  target_network.set_weights((1 - TAU) * np.array(target_network.get_weights()) + TAU * np.array(model.get_weights()))


n: 1 tf.Tensor(0.25, shape=(), dtype=float32)
n: 2 tf.Tensor(0.25, shape=(), dtype=float32)
n: 3 tf.Tensor(0.25, shape=(), dtype=float32)
n: 4 tf.Tensor(0.25, shape=(), dtype=float32)
n: 5 tf.Tensor(0.25, shape=(), dtype=float32)
n: 6 tf.Tensor(0.25, shape=(), dtype=float32)
n: 7 tf.Tensor(0.25, shape=(), dtype=float32)
n: 8 tf.Tensor(0.25, shape=(), dtype=float32)
n: 9 tf.Tensor(0.25, shape=(), dtype=float32)
max_steps=999998
AA tf.Tensor(0.25, shape=(), dtype=float32)
n: 0 tf.Tensor(0.25, shape=(), dtype=float32)


...

/var/folders/mt/0n8phlt50rn7h35xhgh97mw80000gn/T/ipykernel_30589/3025377550.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  target_network.set_weights((1 - TAU) * np.array(target_network.get_weights()) + TAU * np.array(model.get_weights()))


n: 0 tf.Tensor(0.25, shape=(), dtype=float32)
n: 1 tf.Tensor(0.25, shape=(), dtype=float32)
n: 2 tf.Tensor(0.25, shape=(), dtype=float32)
n: 3 tf.Tensor(0.25, shape=(), dtype=float32)
n: 4 tf.Tensor(0.25, shape=(), dtype=float32)
n: 5 tf.Tensor(0.25, shape=(), dtype=float32)
n: 6 tf.Tensor(0.25, shape=(), dtype=float32)
n: 7 tf.Tensor(0.25, shape=(), dtype=float32)
n: 8 tf.Tensor(0.25, shape=(), dtype=float32)
n: 9 tf.Tensor(0.25, shape=(), dtype=float32)
max_steps=999951
AA tf.Tensor(0.25, shape=(), dtype=float32)


/var/folders/mt/0n8phlt50rn7h35xhgh97mw80000gn/T/ipykernel_30589/3025377550.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  target_network.set_weights((1 - TAU) * np.array(target_network.get_weights()) + TAU * np.array(model.get_weights()))


n: 0 tf.Tensor(0.25, shape=(), dtype=float32)
n: 1 tf.Tensor(0.25, shape=(), dtype=float32)
n: 2 tf.Tensor(0.25, shape=(), dtype=float32)
n: 3 tf.Tensor(0.25, shape=(), dtype=float32)
n: 4 tf.Tensor(0.25, shape=(), dtype=float32)
n: 5 tf.Tensor(0.25, shape=(), dtype=float32)
n: 6 tf.Tensor(0.25, shape=(), dtype=float32)
n: 7 tf.Tensor(0.25, shape=(), dtype=float32)
n: 8 tf.Tensor(0.25, shape=(), dtype=float32)
n: 9 tf.Tensor(0.25, shape=(), dtype=float32)


KeyboardInterrupt: 